In [4]:
import os
import glob as gb
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [5]:
training_data = '/kaggle/input/intel-image-classification/seg_train/seg_train/'
testing_data = '/kaggle/input/intel-image-classification/seg_test/seg_test/'
prediction_data = '/kaggle/input/intel-image-classification/seg_pred/seg_pred/'

In [6]:
for folder in os.listdir(training_data):
    files = gb.glob(pathname=str(training_data + folder + '/*.jpg'))
    print(len(files))  
    

In [7]:
for folder in os.listdir(testing_data):
    files = gb.glob(pathname=str(training_data + folder + '/*.jpg'))
    print(len(files))  

In [8]:
files = gb.glob(pathname=str(prediction_data + '/*.jpg'))
print(len(files))  

In [9]:
code = {"buildings" : 0, "forest" : 1,  "glacier" : 2, "mountain" : 3, "sea" : 4, "street" : 5 }

In [10]:
def get_code(name):
    for key, value in code.items():
        if value == name:
            return key

In [11]:
for folder in os.listdir(testing_data):
     print(folder)  

In [12]:
size = []
for folder in os.listdir(training_data):
    files = gb.glob(pathname=str(training_data + folder + '/*.jpg'))
    for file in files:
        size.append(cv2.imread(file).shape)
pd.Series(size).value_counts()

In [13]:
s = 100

In [14]:
x_train = []
y_train = []

In [15]:
for folder in os.listdir(training_data):
    files = gb.glob(pathname=str(training_data + folder + '/*.jpg'))
    for file in files:
        image = cv2.imread(file)
        image_array = cv2.resize(image, (s, s))
        x_train.append(list(image_array))
        y_train.append(code[folder])

In [16]:
x_test = []
y_test = []

In [17]:
for folder in os.listdir(testing_data):
    files = gb.glob(pathname=str(testing_data + folder + '/*.jpg'))
    for file in files:
        image = cv2.imread(file)
        image_array = cv2.resize(image, (s, s))
        x_test.append(list(image_array))
        y_test.append(code[folder])

In [18]:
x_pred = []

In [19]:
files = gb.glob(pathname=str(prediction_data + '/*.jpg'))
for file in files:
    image = cv2.imread(file)
    image_array = cv2.resize(image, (s, s))
    x_pred.append(list(image_array))

In [20]:
x_train = np.array(x_train)
x_train.shape

In [21]:
x_test = np.array(x_test)
x_test.shape

In [22]:
y_train = np.array(y_train)
y_train.shape

In [23]:
y_test = np.array(y_test)
y_test.shape

In [24]:
x_pred = np.array(x_pred)
x_pred.shape

In [25]:
plt.figure(figsize=(20,20))
for n, idx in enumerate(list(np.random.randint(0, len(x_train), 6))):
    plt.subplot(6,6, n+1)
    plt.axis('off')
    plt.imshow(x_train[idx])
    plt.title(get_code(y_train[idx]))

In [26]:
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.layers import Dropout
from keras.models import Sequential

In [27]:
model = Sequential()

In [28]:
model.add(Conv2D(200, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(148, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D((3,3)))
model.add(Conv2D(110, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(66, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(54, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D((3,3)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.43))
model.add(Dense(36, activation='relu'))
model.add(Dropout(0.12))
model.add(Dense(6, activation='softmax'))

In [29]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [30]:
model.summary

In [31]:
x_train = x_train.astype(float)

In [32]:
x_test = x_test.astype(float)

In [33]:
x_pred = x_pred.astype(float)

In [34]:
x_pred[0].shape

In [35]:
fitmodel = model.fit(x_train, y_train, epochs=50, batch_size=60, verbose=1)

In [36]:
loss, acc = model.evaluate(x_test, y_test)

In [39]:
y_pred = model.predict(
    np.array(x_pred[0]).reshape(1, 100, 100, 3)
)
np.argmax(y_pred)

In [49]:
plt.figure()
plt.imshow(
    np.array(x_pred[0]).reshape(100, 100, 3)
)
x_pred[0]